In [45]:
pip install dotenv pypdf langchain langchain-community langchain-openai langchain-pinecone docx2txt tiktoken  -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [47]:
pip install wikipedia -q 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
def load_document(file):
    import os

    name, extension = os.path.splitext(file)

    if extension == ".pdf":
        from langchain_classic.document_loaders import PyPDFLoader

        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain_classic.document_loaders import Docx2txtLoader

        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported")
        return None

    data = loader.load()
    return data


# wikipedia


def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain_classic.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [49]:
def chunk_data(data, chunk_size=256):
    from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [50]:
def print_embedding_cost(texts):
    import tiktoken

    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}")

### Embedding and uploading to a Vector Database (Pinecone)

In [99]:
def insert_or_fetch_embeddings(index_name, chunks):
    import pinecone

    # from langchain_community.vectorstores import Pinecone
    from langchain_pinecone.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec

    pc = pinecone.Pinecone()
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

    if index_name in pc.list_indexes().names():
        print(f"Index {index_name} already exists. Loading embeddings...", end="")
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print("Ok")
    else:
        print(f"Creating index {index_name} and embeddings...", end="")
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        vector_store = Pinecone.from_documents(
            chunks, embeddings, index_name=index_name
        )
        print("Ok")
        return vector_store

In [52]:
def delete_pinecone_index(index_name="all"):
    import pinecone

    pc = pinecone.Pinecone()
    if index_name == "all":
        indexes = pc.list_indexes().names()
        print("Deleting all indexes...")
        for index in indexes:
            pc.delete_index(index)
        print("Ok")
    else:
        print(f"Deleting index {index_name}...", end="")
        pc.delete_index(index_name)
        print("Ok")

In [93]:
def ask_and_get_answer(vector_store, q):
    from langchain_openai import ChatOpenAI
    from langchain_classic.chains.retrieval_qa.base import RetrievalQA

    llm = ChatOpenAI(temperature=1)

    retriever = vector_store.as_retriever(
        search_type="similarity", search_kwargs={"k": 5}
    )

    chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=retriever
    )
    answer = chain.invoke(q)
    return answer


## Running code

In [87]:
data = load_document("pdfs/guia_lgpd.pdf")
# print(data[1].page_content)

print(f"You have {len(data)} pages in your data")

Loading pdfs/guia_lgpd.pdf
You have 69 pages in your data


In [8]:
# data = load_from_wikipedia("Gemini-AI")
# print(data[0].page_content)

In [88]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

929
HISTÓRICO DE VERSÕES
DATA VERSÃO DESCRIÇÃO AUTOR
23/03/2020 1.0 Primeira versão do Guia de Boas Práticas. Equipe Técnica de 
Elaboração
14/08/2020 2.0
Segunda versão do Guia de Boas Práticas 
considerando as sugestões enviadas pelo:


In [96]:
print_embedding_cost(chunks)

Total Tokens: 52383
Embedding Cost in USD: 0.001048


In [102]:
delete_pinecone_index()

Deleting all indexes...
Ok


In [103]:
index_name = "askadocument"
vector_store = insert_or_fetch_embeddings(index_name, chunks)

Creating index askadocument and embeddings...Ok


In [104]:
q = "Sobre o que o documento fala?"
answer = ask_and_get_answer(vector_store, q)
print(answer["result"])

O documento fala sobre a Lei Geral de Proteção de Dados (LGPD) e como as obrigações de transparência ativa e os meios de acesso à informação se relacionam com as operações de tratamento de dados pessoais, em conjunto com a gestão de documentos ao longo do ciclo de vida do documento. Além disso, menciona o ciclo de vida de tratamento dos dados pessoais, padrões e frameworks de segurança da informação, e faz referências a normas e regulamentos relacionados à gestão de documentos e as atividades do Poder Executivo federal e do Centro de Comando e Controle (CCD).


In [68]:
import time

i = 1
print("Write Quit or Exit to quit.")
while True:
    q = input(f"Question #{i}: ")
    i = i + 1
    if q.lower() in ["quit", "exit"]:
        print("Quitting... bye")
        time.sleep(2)
        break

    answer = ask_and_get_answer(vector_store, q)
    print(f"\nAnswer: {answer['result']}")
    print(f"\n {'-' * 50} \n")

Write Quit or Exit to quit.

Answer: A ePING é um conjunto de padrões de interoperabilidade de Governo Eletrônico que define premissas, políticas e especificações técnicas para a utilização de Tecnologia de Informação e Comunicação (TIC) visando a interconexão, segurança, meios de acesso, organização e intercâmbio de informações, e áreas de integração para o Governo Eletrônico. Esses padrões visam garantir a interoperabilidade dos serviços de Governo Eletrônico com os demais Poderes, esferas de governo e a sociedade em geral. Mais informações podem ser encontradas no site http:/ /eping.governoeletronico.gov.br/

 -------------------------------------------------- 

Quitting... bye


## Chroma DB

In [69]:
pip install chromadb -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [70]:
def create_embeddings_chroma(chunks, persist_directory="./chroma_db"):
    from langchain_classic.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)
    vector_store = Chroma.from_documents(
        chunks, embeddings, persist_directory=persist_directory
    )
    return vector_store


def load_embeddings_chroma(persist_directory="./chroma_db"):
    from langchain_classic.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)
    vector_store = Chroma(
        persist_directory=persist_directory, embedding_function=embeddings
    )
    return vector_store


In [71]:
data = load_document("pdfs/lgpd.pdf")
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)

Loading pdfs/lgpd.pdf


In [72]:
q = "Sobre o que o documento fala:"

answer = ask_and_get_answer(vector_store, q)
print(answer["result"])

O documento em questão parece abordar a proteção de dados pessoais e a privacidade, incluindo a necessidade de relatório de impacto à proteção de dados pessoais e a comunicação de incidentes de segurança que possam acarretar riscos ou danos aos titulares de dados.


In [74]:
db = load_embeddings_chroma()
q = "Quem criou a LGPD?"

answer = ask_and_get_answer(vector_store, q)
print(answer["result"])

A Lei Geral de Proteção de Dados Pessoais (LGPD) foi criada pelo Congresso Nacional do Brasil e foi sancionada pelo Presidente da República.


## Adding Memory (Chat history)

In [105]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains.conversational_retrieval.base import (
    ConversationalRetrievalChain,
)
from langchain_classic.memory import ConversationBufferMemory

llm = ChatOpenAI(model="gpt-5", temperature=0)
retriever = vector_store.as_retriever(search_type="similarity", search_kwards={"k": 5})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

crc = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory, chain_type="stuff", verbose=True
)

In [106]:
def ask_question(q, chain):
    result = chain.invoke({"question": q})
    return result

In [107]:
data = load_document("pdfs/lgpd.pdf")
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)

Loading pdfs/lgpd.pdf


In [ ]:
q = "Que ano foi criada a lei?"
result = ask_question(q, crc)
print(result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Acesso http:/ /www.planalto.gov.br/ccivil_03/LEIS/L9507.htm
Data do ato 29/01/1999
Data da publicação no DOU 01/02/1999 e retificado em 11/03/1999
Vigência (vacatio legis) Art. 70. Esta Lei entra em vigor na data de sua publicação.

dispositivos da Lei nº 8.159, de 8 de janeiro de 1991; e dá outras providências.
Data do ato 12/11/1997
Data da publicação no DOU 13/11/1997
Vigência (vacatio legis) Art. 22. Esta Lei entra em vigor na data de sua publicação.

dispositivos da Lei no 8.159, de 8 de janeiro de 1991; e dá outras providências. Disponível 
em: < http:/ /www.planalto.gov.br/ccivil_03/_Ato2011-2014/2011/Lei/L12527.htm#art1>. 
Acesso em: 09 abr. 2019.

Acesso http:/ /www.planalto.gov.br/ccivil_03/

In [ ]:
q = "Multiplique por 2"
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Quem criou a lei?
Assistant: Se você se refere à LGPD (Lei nº 13.709/2018), ela foi aprovada pelo Congresso Nacional e sancionada pelo Presidente da República Michel Temer em 14/08/2018. O texto resultou da consolidação de projetos debatidos no Congresso (como o PL 4.060/2012 e o PL 5.276/2016, este último do Poder Executivo).
Follow Up Input: Multiplique o ano da lei geral do processo administrativo por 2
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
25Lei no 13.709/2018
1997 (Lei 

In [85]:
for item in result["chat_history"]:
    print(item)

content='Quem criou a lei?' additional_kwargs={} response_metadata={}
content='Se você se refere à LGPD (Lei nº 13.709/2018), ela foi aprovada pelo Congresso Nacional e sancionada pelo Presidente da República Michel Temer em 14/08/2018. O texto resultou da consolidação de projetos debatidos no Congresso (como o PL 4.060/2012 e o PL 5.276/2016, este último do Poder Executivo).' additional_kwargs={} response_metadata={}
content='Multiplique o ano da lei geral do processo administrativo por 2' additional_kwargs={} response_metadata={}
content='1999 × 2 = 3998.' additional_kwargs={} response_metadata={}
content='Multiplique por 2' additional_kwargs={} response_metadata={}
content='7996' additional_kwargs={} response_metadata={}
